![alt text](./img/header.png)

# Saltwater Intrusion Exercise - Now using SWI


In [ ]:
# Setup the python environment
%matplotlib inline
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import flopy
import config

## Exercise B-SWI $-$ Create a equivalent model to exSEAWAT_B using the SWI2 Package

Based on the geometry of the system, the pre-withdrawal conditions can be obtained by running a 2D cross-section model.  The resulting heads and salinities can then be used as initial conditions for the 3D model to evaluate interface movement in response to pumping.  Because 3D saltwater intrusion models can take a long time, across-sectional model is developed first using one row, 100 columns, and 3 model layers. The model has already been constructed for you.

This exercise was designed such that a 2D model could be used to represent pre-withdrawal conditions.  If the hydrogeology were to vary in the north-south direction, or if an irregular boundary were to exist, the problem of obtaining equilibrium conditions would be more difficult.

In [ ]:
# Data path
datapth = os.path.join('data', 'exSEAWAT_B-SWI')

# Grid information
nlay = 3
nrow = 1
ncol = 100
delr = 100.
delc = 100.

# Temporal discretization
nper = 1
nstp = 10000
perlen = 1000000.

# Top
fname = os.path.join(datapth, 'top.txt')
top = np.loadtxt(fname).reshape((nrow, ncol))

# Bottom
fname = os.path.join(datapth, 'botm.txt')
botm = np.loadtxt(fname).reshape((nlay, nrow, ncol))

# Ibound
fname = os.path.join(datapth, 'ibound.txt')
ibound = np.loadtxt(fname).reshape((nlay, nrow, ncol))

# Hydraulic properties
fname = os.path.join(datapth, 'hk.txt')
hk = np.loadtxt(fname).reshape((nlay, nrow, ncol))
fname = os.path.join(datapth, 'vk.txt')
vk = np.loadtxt(fname).reshape((nlay, nrow, ncol))
ss = 1.e-5
sy = 0.2

In [ ]:
# Recharge
rech = 25.4 / 100. / 365.  # convert cm/yr to m/day

# Build ghbs
# Determine the number of ghbs
nghb = 0
for k in range(nlay):
    for j in range(ncol):
        if ibound[k, 0, j] == 2:
            nghb += 1
# Empty ghb recarray
ghb_data = flopy.modflow.ModflowGhb.get_empty(ncells=nghb)
# Fill the recarray
idx = 0
for k in range(nlay):
    for j in range(ncol):
        if ibound[k, 0, j] == 2:
            if k == 0:
                z1 = min(0., top[0, j])
            else:
                z1 = botm[k-1, 0, j]
            hfw = 0.025 * (-z1)
            d = (z1 - botm[k, 0, j])
            cond = hk[k, 0, j] * d * delr / (0.5 * delc)
            ghb_data['k'][idx] = k
            ghb_data['i'][idx] = 0
            ghb_data['j'][idx] = j
            ghb_data['bhead'][idx] = hfw
            ghb_data['cond'][idx] = cond
            idx += 1
#print(ghb_data.dtype)
#print(ghb_data)

In [ ]:
# Swi input data
isource = np.zeros((nlay, nrow, ncol), dtype=np.int)
for k in range(nlay):
    for j in range(ncol):
        if ibound[k, 0, j] == 2:
            isource[k, 0, j] = -2

# assume freshwater to coastline and saltwater beyond
ipos = 0
for j in range(ncol):
    if top[0, j] < 2.5:
        ipos = j
        break
zeta = np.ones((nlay, nrow, ncol)) * -250.
zeta[:, :, ipos:] = 2.5

In [ ]:
# Build the flopy SEAWAT model
model_ws = os.path.join('work', 'exSEAWAT_B-SWI')
modelname = 'b1-swi'
m = flopy.modflow.Modflow(modelname, model_ws=model_ws, exe_name=config.mfexe)
dis = flopy.modflow.ModflowDis(m, nlay=3, nrow=nrow, ncol=ncol, delr=delr, delc=delc, 
                               top=top, botm=botm, perlen=perlen, nstp=nstp)
bas = flopy.modflow.ModflowBas(m, ibound, strt=0.)
lpf = flopy.modflow.ModflowLpf(m, laytyp=1, hk=hk, vka=vk, ss=ss, sy=sy)
rch = flopy.modflow.ModflowRch(m, rech=rech)
ghb = flopy.modflow.ModflowGhb(m, stress_period_data=ghb_data)
swi = flopy.modflow.ModflowSwi2(m, ssz=0.2, zeta=[zeta], isource=isource, nsrf=1, nobs=3, 
                                obslrc=[(0, 0, ncol-28), (0, 0, ncol-27), 
                                        (0, 0, ncol-26), (2, 0, ncol-12),
                                        (2, 0, ncol-10), (2, 0, ncol-8)], 
                                obsnam=['L1C73', 'L1C74', 
                                        'L1C75', 'L3C89', 
                                        'L3C91', 'L3C93'], 
                                iswiobs=1053, iswizt=55)
oc = flopy.modflow.ModflowOc(m, save_every=10)
pcg = flopy.modflow.ModflowPcg(m, hclose=1.e-3, rclose=1e-1)

In [ ]:
# Make plot of the grid
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row':0})
linecollection = mm.plot_grid()
patchcollection = mm.plot_ibound()

In [ ]:
# Make color flood plot of hydraulic conductivity
f = plt.figure(figsize=(15, 5))
ax = f.add_subplot(1, 1, 1)
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row':0})
hkpatchcollection = mm.plot_array(np.log(m.lpf.hk.array), cmap='viridis')
linecollection = mm.plot_grid()
patchcollection = mm.plot_ibound()
cb = plt.colorbar(hkpatchcollection)

In [ ]:
# write the input files
m.write_input()

In [ ]:
# run the model
success, buff = m.run_model(silent=True)
print(success)

In [ ]:
# Extract zeta surface
fname = os.path.join(model_ws, 'b1-swi.zta')
zobj = flopy.utils.binaryfile.CellBudgetFile(fname)
#print(zobj.list_records())
kk = zobj.get_kstpkper()
zeta = zobj.get_data(text='ZETASRF  1', kstpkper=kk[-1])[0]

In [ ]:
# import seawat model 
swt_import = False
try:
    swt_ws = os.path.join('work', 'exSEAWAT_B')
    swt_name = 'b1'
    ms = flopy.seawat.Seawat.load(swt_name+'.nam', model_ws=swt_ws)
    # Extract salinity
    fname = os.path.join(swt_ws, 'MT3D001.UCN')
    ucnobj = flopy.utils.binaryfile.UcnFile(fname)
    times = ucnobj.get_times()
    conc = ucnobj.get_data(totim=times[-1])
    conc[np.where(ms.bas6.ibound.array != 1)] = np.nan    
    swt_import = True
except:
    swt_ws = False
    print('Houston we have a problem')
    pass

### Plot the zeta surface

Plot the zeta surface at the end of the simulation and compare it to the SEAWAT results for the same problem.

In [ ]:
# Make plot of the simulated zeta surface and compare to the SEAWAT results
f = plt.figure(figsize=(15, 10))
ax = f.add_subplot(2, 1, 1)
mm = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row':0})
mm.plot_fill_between(zeta);
linecollection = mm.plot_grid()
ax.set_title('SWI Package')
if swt_ws:
    ax2 = f.add_subplot(2, 1, 2)
    mm2 = flopy.plot.ModelCrossSection(ax=ax2, model=ms, line={'row':0})
    cpatchcollection = mm2.plot_array(conc, vmin=0.1, vmax=35, edgecolor='k')
    mm2.contour_array(conc, levels=[35./2.], colors='red', linewidths=4.)
    ax2.set_title('SEAWAT')

### B1-SWI Questions
 
1. Do the differences between SWI and SEAWAT make sense given the assumptions used in SWI?

### Determine if the zeta surface is stable

Zeta observations can be used to determine if the the zeta surface is stable.

In [ ]:
fn = os.path.join(model_ws, 'b1-swi.zobs.out')
zobs = np.genfromtxt(fn, names=True)
print(zobs.dtype)

In [ ]:
fig_size=(15, len(zobs.dtype.names[1:])*2.5)
fig,ax = plt.subplots(len(zobs.dtype.names[1:]),1, sharex=True, figsize=fig_size)
for idx, cnam in enumerate(zobs.dtype.names[1:]):
    ax[idx].plot(zobs['TOTIM'], zobs[cnam], label=cnam)
    ax[idx].legend();

### Animate results

In [ ]:
import time
from IPython.display import clear_output, display
animate = False # set to True to run the animation
if animate:
    f = plt.figure(figsize=(15, 5))
    ax = plt.subplot(1, 1, 1)
    for i, t in enumerate(kk):
        zeta = zobj.get_data(text='ZETASRF  1', kstpkper=t)[0]
        ax.set_title('kstp {:05d}/{:05d}'.format(t[0]+1, kk[-1][0]+1))
        xs = flopy.plot.ModelCrossSection(ax=ax, model=m, line={'row': 0})
        xs.plot_fill_between(zeta)
        time.sleep(0.001)
        clear_output(True)
        display(f)
        ax.cla()
    plt.close('all')
    print('Done.');

### B1-SWI Questions
 
1. Based on the zeta observations plots and the animation above, has equilibrium been reached and are the zeta elevations stable?
2. Determine the length of time it takes to achieve a stable zeta surface by rerunning the simulation using different values for `perlen` and `nstp`.
3. What are some options for reducing the small oscillations seen in a few of the zeta observations?